In [1]:
import $exec.spark_functionality

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
23/09/07 09:50:03 WARN Utils: Your hostname, lm3-523-28823.local resolves to a loopback address: 127.0.0.1; using 10.112.14.25 instead (on interface en0)
23/09/07 09:50:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Getting spark JARs


23/09/07 09:50:04 INFO log: Logging initialized @3341ms to org.eclipse.jetty.util.log.Slf4jLog
23/09/07 09:50:04 INFO Server: jetty-9.4.49.v20220914; built: 2022-09-14T01:07:36.601Z; git: 4231a3b2e4cb8548a412a789936d640a97b1aa0a; jvm 17.0.8.1+1
23/09/07 09:50:04 INFO AbstractConnector: Started ServerConnector@61686601{HTTP/1.1, (http/1.1)}{127.0.0.1:54499}
23/09/07 09:50:04 INFO Server: Started @3386ms


Creating SparkSession


23/09/07 09:50:04 INFO SparkContext: Running Spark version 3.4.1
23/09/07 09:50:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/07 09:50:04 INFO ResourceUtils: ==============================================================
23/09/07 09:50:04 INFO ResourceUtils: No custom resources configured for spark.driver.
23/09/07 09:50:04 INFO ResourceUtils: ==============================================================
23/09/07 09:50:04 INFO SparkContext: Submitted application: b1651b26-88ec-4879-9c1e-92411be74a48
23/09/07 09:50:04 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
23/09/07 09:50:04 INFO ResourceProfile: Limiting resource is cpu
23/09/07 09:50:04 INFO

Spark UI

23/09/07 09:50:07 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


import $exec.$

# 2. Defragment overlapping pieces to one id

Create an initial defragmentation of the piece_ids by mapping all pieces that overlap sufficiently in a given text document to one id

In [2]:
import spark.implicits._
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, ArrayType}
import org.apache.spark.sql.catalyst.encoders.RowEncoder

val input_type = StructType(Seq(
    StructField("trs_start",IntegerType),
    StructField("trs_end",IntegerType),
    StructField("piece_id",IntegerType)
))

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql.expressions.Aggregator

object GetPieceIdMapping extends Aggregator[Row, ArrayBuffer[(Int,Int,Int)], Int] {
  import org.apache.spark.sql.catalyst.encoders.ExpressionEncoder

  // A zero value for this aggregation. Should satisfy the property that any b + zero = b
  def zero: ArrayBuffer[(Int,Int,Int)] = ArrayBuffer()
  // Combine two values to produce a new value. For performance, the function may modify `buffer`
  // and return it instead of constructing a new object
  def reduce(to_check: ArrayBuffer[(Int,Int,Int)], row: Row): ArrayBuffer[(Int,Int,Int)] = {
      val t_start = row.getInt(0)
      // check existing buffer for the last piece that is 180 characters 
      // before the current start offset
      val idx = to_check.indexWhere(_._1 >= t_start - 180)
      // if 0 then clear the buffer
      // otherwise remove unsuitable pieces
      if (idx < 0) to_check.clear() else to_check.remove(0, idx)
      // add the current piece to the end of buffer
      to_check += ((t_start,row.getInt(1),row.getInt(2)))
      to_check
  }
  // Merge two intermediate values
  def merge(b1: ArrayBuffer[(Int,Int,Int)], b2: ArrayBuffer[(Int,Int,Int)]): ArrayBuffer[(Int,Int,Int)] = {
//      b1 ++= b2 & sort
      throw new UnsupportedOperationException()
  }
  // Transform the output of the reduction
  def finish(to_check: ArrayBuffer[(Int,Int,Int)]): Int = {
      val (t_start, t_end, piece_id) = to_check.last
      to_check.find(r => {
          val limit = math.min(
              math.max(
                  math.min(t_end - t_start, r._2 - r._1)
                  /4, 10),
              180)
          math.abs(r._1 - t_start) <= limit && math.abs(r._2 - t_end) <= limit
      }).get._3 // get the id of the first piece that is valid
  }
  // Specifies the Encoder for the intermediate value type
  def bufferEncoder: Encoder[ArrayBuffer[(Int,Int,Int)]] = ExpressionEncoder()
  // Specifies the Encoder for the final output value type
  def outputEncoder: Encoder[Int] = Encoders.scalaInt
}

// Register the function to access it
spark.udf.register("get_piece_id_mapping", functions.udaf(GetPieceIdMapping,RowEncoder(input_type)))

import spark.implicits._
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, ArrayType}
import org.apache.spark.sql.catalyst.encoders.RowEncoder
input_type: StructType = StructType(
  StructField("trs_start", IntegerType, true, {}),
  StructField("trs_end", IntegerType, true, {}),
  StructField("piece_id", IntegerType, true, {})
)
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql.expressions.Aggregator
defined object GetPieceIdMapping
res2_7: expressions.UserDefinedFunction = UserDefinedAggregator(
  ammonite.$sess.cell2$Helper$GetPieceIdMapping$@5e8b5dea,
  ExpressionEncoder(
    If(
      IsNull(
        BoundReference(0, ObjectType(interface org.apache.spark.sql.Row), true)
      ),
      Literal(
        null,
        StructType(
          StructField("trs_start", IntegerType, true, {}),
          StructField("trs_end", IntegerType, true, {}),
          StructField("piece_id", IntegerType, true, {})
        )
      ),
      CreateNamedStru

In [3]:
val orig_pieces = get_a3s("orig_pieces")

parquet at spark_functionality.sc:134 
 (kill)

0 / 1

orig_pieces: DataFrame = [piece_id: bigint, trs_id: bigint ... 2 more fields]

In [4]:
val piece_id_mappings = materialise_a3s_if_not_exists(
    "piece_id_mappings_tmp", 
    spark.sql("""
    SELECT 
        piece_id AS orig_piece_id,
        get_piece_id_mapping(trs_start,trs_end,piece_id) 
            OVER (PARTITION BY trs_id ORDER BY trs_start, piece_id) 
            AS defrag_mapping
    FROM orig_pieces
    """))

parquet at spark_functionality.sc:138 
 (kill)

0 / 14

parquet at spark_functionality.sc:138 
 (kill)

0 / 24

parquet at spark_functionality.sc:134 
 (kill)

0 / 1

piece_id_mappings: DataFrame = [orig_piece_id: bigint, defrag_mapping: int]

In [5]:
materialise_row_numbers(
  "defrag_id_mapping",
  spark.sql("""
    SELECT DISTINCT defrag_mapping
    FROM piece_id_mappings_tmp
    ORDER BY defrag_mapping
    """),
  "defrag_piece_id"
)

rdd at spark_functionality.sc:176 
 (kill)

0 / 12

rdd at spark_functionality.sc:176 
 (kill)

0 / 13

rdd at spark_functionality.sc:176 
 (kill)

...

0 / 12

parquet at spark_functionality.sc:138 
 (kill)

0 / 13

parquet at spark_functionality.sc:134 
 (kill)

0 / 1

In [6]:
val updated_piece_id_mappings = materialise_a3s("piece_id_mappings", spark.sql("""
SELECT orig_piece_id, defrag_piece_id FROM piece_id_mappings_tmp
INNER JOIN defrag_id_mapping USING(defrag_mapping)
"""))
// delete the temporary mapping dataframes
delete_s3("piece_id_mappings_tmp")
delete_s3("defrag_id_mapping")

parquet at spark_functionality.sc:138 
 (kill)

0 / 12

parquet at spark_functionality.sc:138 
 (kill)

0 / 13

parquet at spark_functionality.sc:138 
 (kill)

0 / 13

parquet at spark_functionality.sc:134 
 (kill)

0 / 1

updated_piece_id_mappings: DataFrame = [orig_piece_id: bigint, defrag_piece_id: bigint]